# Classification

In [ ]:
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection, preprocessing, metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
dataset_adult = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data")
dataset_adult.columns = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","income"]

In [ ]:
dataset_adult.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


**Data Preproccesing**

In [ ]:
df_obj = dataset_adult.select_dtypes(['object'])
dataset_adult[df_obj.columns] = df_obj.apply(lambda x:x.str.strip())

In [ ]:
dataset_adult['workclass'] = dataset_adult['workclass'].replace('?',np.nan)
dataset_adult['occupation'] = dataset_adult['occupation'].replace('?',np.nan)
dataset_adult['native-country'] = dataset_adult['native-country'].replace('?',np.nan)

In [ ]:
dataset_adult.dropna(inplace=True)

In [ ]:
to_replace = list(dataset_adult['marital-status'].unique())
to_replace.remove('Never-married')
dataset_adult.replace('Never-married','Single',inplace=True)
dataset_adult.replace(to_replace,'Married',inplace=True)
dataset_adult['marital-status'].value_counts(normalize=True)*100

Married    67.756374
Single     32.243626
Name: marital-status, dtype: float64

In [ ]:
labelEncoder = preprocessing.LabelEncoder()
category_col =['race','marital-status', 'sex', 'income'] 

for col in category_col:
    dataset_adult[col] = labelEncoder.fit_transform(dataset_adult[col])

category_col_1 =['workclass', 'education', 'occupation',
               'relationship','native-country'] 
df_2 = pd.get_dummies(dataset_adult, columns=category_col_1, drop_first=True)

In [ ]:
dataframe=df_2.drop('fnlwgt',1)
dataframe =dataframe[[c for c in dataframe if c not in ['income']] + ['income']]
dataframe.head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,age,education-num,marital-status,race,sex,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Private,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income
0,50,13,0,4,1,0,0,13,0,0,...,0,0,0,0,0,0,1,0,0,0
1,38,9,0,4,1,0,0,40,0,1,...,0,0,0,0,0,0,1,0,0,0
2,53,7,0,2,1,0,0,40,0,1,...,0,0,0,0,0,0,1,0,0,0
3,28,13,0,2,0,0,0,40,0,1,...,0,0,0,0,0,0,0,0,0,0
4,37,14,0,4,0,0,0,40,0,1,...,0,0,0,0,0,0,1,0,0,0
5,49,5,0,2,0,0,0,16,0,1,...,0,0,0,0,0,0,0,0,0,0
6,52,9,0,4,1,0,0,45,0,0,...,0,0,0,0,0,0,1,0,0,1
7,31,14,1,4,0,14084,0,50,0,1,...,0,0,0,0,0,0,1,0,0,1
8,42,13,0,4,1,5178,0,40,0,1,...,0,0,0,0,0,0,1,0,0,1
9,37,10,0,2,1,0,0,80,0,1,...,0,0,0,0,0,0,1,0,0,1


In [ ]:
X = dataframe.iloc[:, 0:-1].values
y = dataframe.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# tạo mô hình tuần tự
classifier = Sequential() 

classifier.add(Dense(6, activation = 'relu', input_dim = 87))
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dense(1, activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 50)

Epoch 1/50
2413/2413 [==============================] - 5s 2ms/step - loss: 0.4111 - accuracy: 0.8021
Epoch 2/50
2413/2413 [==============================] - 4s 2ms/step - loss: 0.3351 - accuracy: 0.8430
Epoch 3/50
2413/2413 [==============================] - 4s 2ms/step - loss: 0.3270 - accuracy: 0.8470
Epoch 4/50
2413/2413 [==============================] - 4s 2ms/step - loss: 0.3243 - accuracy: 0.8480
Epoch 5/50
2413/2413 [==============================] - 4s 2ms/step - loss: 0.3221 - accuracy: 0.8479
Epoch 6/50
2413/2413 [==============================] - 4s 2ms/step - loss: 0.3206 - accuracy: 0.8500
Epoch 7/50
2413/2413 [==============================] - 4s 2ms/step - loss: 0.3186 - accuracy: 0.8509
Epoch 8/50
2413/2413 [==============================] - 4s 2ms/step - loss: 0.3176 - accuracy: 0.8508
Epoch 9/50
2413/2413 [==============================] - 4s 2ms/step - loss: 0.3171 - accuracy: 0.8501
Epoch 10/50
2413/2413 [==============================] - 4s 2ms/step - loss: 0.316

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
print ("\nClassification Report\n")
print (classification_report(y_test, y_pred))


Classification Report

              precision    recall  f1-score   support

           0       0.88      0.93      0.91      4570
           1       0.74      0.61      0.67      1463

    accuracy                           0.85      6033
   macro avg       0.81      0.77      0.79      6033
weighted avg       0.85      0.85      0.85      6033



# Regression

In [ ]:
ff_data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv")
ff_data.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [ ]:
ff_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       517 non-null    int64  
 1   Y       517 non-null    int64  
 2   month   517 non-null    object 
 3   day     517 non-null    object 
 4   FFMC    517 non-null    float64
 5   DMC     517 non-null    float64
 6   DC      517 non-null    float64
 7   ISI     517 non-null    float64
 8   temp    517 non-null    float64
 9   RH      517 non-null    int64  
 10  wind    517 non-null    float64
 11  rain    517 non-null    float64
 12  area    517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [ ]:
ff_data.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,4.669246,4.299807,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292
std,2.313778,1.229900,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818
min,1.000000,2.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000
25%,3.000000,4.000000,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000
50%,4.000000,4.000000,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000
75%,7.000000,5.000000,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000
max,9.000000,9.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000


In [ ]:
ff_data['Log-area']=np.log10(ff_data['area']+1)

In [ ]:
enc = LabelEncoder()
enc.fit(ff_data['month'])
ff_data['month_encoded']=enc.transform(ff_data['month'])
ff_data.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,Log-area,month_encoded
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,0.0,7
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0.0,10
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0.0,10
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,0.0,7
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0.0,7


In [ ]:
enc.fit(ff_data['day'])
ff_data['day_encoded']=enc.transform(ff_data['day'])
ff_data.head(10)

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,Log-area,month_encoded,day_encoded
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,0.0,7,0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0.0,10,5
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0.0,10,2
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,0.0,7,0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0.0,7,3
5,8,6,aug,sun,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.0,0.0,1,3
6,8,6,aug,mon,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.0,0.0,1,1
7,8,6,aug,mon,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,0.0,0.0,1,1
8,8,6,sep,tue,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,0.0,0.0,11,5
9,7,5,sep,sat,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,0.0,0.0,11,2


In [ ]:
ff_data.drop(['month','day'],axis=1,inplace=True)

In [ ]:
ff_data.head()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,Log-area,month_encoded,day_encoded
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,0.0,7,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0.0,10,5
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0.0,10,2
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,0.0,7,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0.0,7,3


In [ ]:
X = ff_data.drop(['area','Log-area'],axis=1).values
y = ff_data['Log-area'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
model = Sequential()
model.add(Dense(500, input_dim = X_train.shape[1], activation= "relu"))
model.add(Dense(100, activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 500)               6500      
                                                                 
 dense_4 (Dense)             (None, 100)               50100     
                                                                 
 dense_5 (Dense)             (None, 50)                5050      
                                                                 
 dense_6 (Dense)             (None, 1)                 51        
                                                                 
Total params: 61,701
Trainable params: 61,701
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

In [ ]:
data=X_train
target = y_train
model.fit(data, target, epochs=50, batch_size=10)

Epoch 1/50
37/37 [==============================] - 1s 3ms/step - loss: 144.5164 - mean_squared_error: 144.5164
Epoch 2/50
37/37 [==============================] - 0s 3ms/step - loss: 1.4738 - mean_squared_error: 1.4738
Epoch 3/50
37/37 [==============================] - 0s 2ms/step - loss: 0.6425 - mean_squared_error: 0.6425
Epoch 4/50
37/37 [==============================] - 0s 3ms/step - loss: 0.7491 - mean_squared_error: 0.7491
Epoch 5/50
37/37 [==============================] - 0s 2ms/step - loss: 0.5512 - mean_squared_error: 0.5512
Epoch 6/50
37/37 [==============================] - 0s 2ms/step - loss: 0.6443 - mean_squared_error: 0.6443
Epoch 7/50
37/37 [==============================] - 0s 2ms/step - loss: 0.5559 - mean_squared_error: 0.5559
Epoch 8/50
37/37 [==============================] - 0s 2ms/step - loss: 0.5842 - mean_squared_error: 0.5842
Epoch 9/50
37/37 [==============================] - 0s 2ms/step - loss: 0.7776 - mean_squared_error: 0.7776
Epoch 10/50
37/37 [=====

In [ ]:
a=model.predict(X_test)
print("RMSE for Deep Network:",np.sqrt(np.mean((y_test-a.reshape(a.size,))**2)))

RMSE for Deep Network: 0.724915240847617
